# Notebook for morphological operations

## Make the import

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

### Read and display the image

In [ ]:
data_root = "Images/"

# Read in grayscale
image = cv2.imread(os.path.join(data_root, "rice.jpg"), cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(image, cmap="gray")

### Threshold the image into a binary image

In [ ]:
def otsu_parts(img, n, direction):
    
    out = []
    
    if direction == 'horizontal':
        l = img.shape[0]
        h = np.linspace(0, l, n+1).astype('int')
        
        for i in range(n):
            ret, th = cv2.threshold(img[h[i]: h[i+1], :], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            out.append(th)
    else:
        l = img.shape[1]
        h = np.linspace(0, l, n+1).astype('int')
        
        for i in range(n):
            ret, th = cv2.threshold(img[:, h[i]: h[i+1]], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            out.append(th)
    
    
    return np.concatenate(out, axis=0)

image_thresh = otsu_parts(image, 5, 'horizontal')

In [ ]:
# _, image_thresh = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)
plt.figure(figsize=(15, 10))
plt.imshow(image_thresh, cmap="gray")

### Implementation of an 7x7 erosion operator

In [ ]:
pad=2
padded = np.pad(image_thresh, ((pad, pad), (pad, pad)), 'constant', constant_values=255) # 255 is the default value for erosion, 0 is for dilation

h, w = image_thresh.shape

new_image = np.zeros_like(image_thresh)

struct_elem = 255*np.ones((2*pad+1, 2*pad+1))

for i in range(pad, h+pad):
    for j in range(pad, w+pad):
        if (padded[i-pad:i+pad+1, j-pad:j+pad+1] == struct_elem).all():
            new_image[i-pad, j-pad] = 255

In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(new_image, cmap="gray")

### Opencv version

In [ ]:
kernel = np.ones((5, 5), np.uint8)

eroded = cv2.erode(image_thresh, kernel, iterations=1)
plt.figure(figsize=(15, 10))
plt.imshow(eroded, cmap="gray")

### Check that the output is the same

In [ ]:
print((new_image == eroded).all())

### Other implementation

In [ ]:
a = cv2.erode(image_thresh, np.ones((5, 5), np.uint8), iterations=1)
b = cv2.erode(image_thresh, np.ones((3, 3), np.uint8), iterations=2)
c = cv2.morphologyEx(image_thresh, cv2.MORPH_ERODE, kernel=np.ones((5, 5), np.uint8), iterations=1)

print((a == b).all(), (a == c).all())

### Dilation with opencv

In [ ]:
dilated = cv2.dilate(image_thresh, kernel, iterations=1)
plt.figure(figsize=(15, 10))
plt.imshow(dilated, cmap="gray")

### Opening with opencv

In [ ]:
opened = cv2.morphologyEx(image_thresh, cv2.MORPH_OPEN, kernel=kernel)
plt.figure(figsize=(15, 10))
plt.imshow(opened, cmap="gray")

### Closing with opencv

In [ ]:
closed = cv2.morphologyEx(image_thresh, cv2.MORPH_CLOSE, kernel=kernel)
plt.figure(figsize=(15, 10))
plt.imshow(closed, cmap="gray")

In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(cv2.erode(cv2.dilate(image_thresh, kernel, iterations=1), kernel, iterations=1), cmap="gray")

### Closing is the cascade of a dilation followed by erosion

In [ ]:
print((closed == cv2.erode(cv2.dilate(image_thresh, kernel, iterations=1), kernel, iterations=1)).all())

In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(dilated - eroded, cmap="gray")